In [2]:
import numpy as np

In [11]:
M = np.linspace(1,8,8).reshape(2,2,2).astype(int)
M

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

In [12]:
ind = np.array([1,0])
ind

array([1, 0])

In [15]:
M[0,:,1]

array([2, 4])

In [10]:
M

array([[2, 1, 3],
       [5, 4, 6],
       [8, 7, 9]])